In [9]:
import pandas as pd
import csv
from glob import glob
from array import array
from pathlib import Path
import ROOT



In [21]:
# .txt -> .csv 

absolute_path_only = '/media/droc/Seagate/PHYS-461/4-hall_effect/data/' #This will need to change depending on the path to data
input_files = glob(absolute_path_only + '/*.txt')
# print(input_files)

base_csv = []
for file in input_files:
    base = Path(file).stem
    base_csv.append(base + '.csv')

input_path = []
output_path = []

for i in range(len(base_csv)):
    input_file_path = input_files[i]
    output_file_path = absolute_path_only + '/' + base_csv[i]

    input_path.append(input_file_path)
    output_path.append(output_file_path)
    
    # print(output_file_path)

    with open(input_file_path, 'r') as txt_file, open(output_file_path, 'w', newline='') as csv_file:
        reader = txt_file.readlines()
        writer = csv.writer(csv_file)
        # print(writer)

        for line in reader:
            
            
            row = line.split()
            writer.writerow(row)
            

In [22]:
# .csv -> .root

def clean_data(df):
        for col in df.columns:
            # Convert all values in the column to float, set non-convertible values to NaN
            df[col] = pd.to_numeric(df[col], errors='coerce')
        return df


out_root = []
for file in output_path:

    df = pd.read_csv(file)

    file_out_root = file.replace('.csv','.root')

    out_root.append(file_out_root)

    root_file = ROOT.TFile(file_out_root, "RECREATE")
    tree = ROOT.TTree("data", "Data from CSV")

    branches = {}

    df = clean_data(df)

    for column in df.columns:
        branches[column] = array('f', [0])  # Create float arrays for each column
        tree.Branch(column, branches[column], f'{column}/F')

    for index, row in df.iterrows():
        for column in df.columns:
            branches[column][0] = row[column]
        tree.Fill()

    tree.Write()
    root_file.Close()

    # print(root_file)

In [17]:
file = ROOT.TFile(out_root[0]) 

# print(file)

tree = file.Get("data")

for entry in tree:
    # print(entry)
    print(entry.stddev) 

file.Close()

0.00044721001177094877
0.0005477200029417872
0.00044721001177094877
0.0
0.00044721001177094877
0.0005477200029417872
0.00044721001177094877
0.0005477200029417872
0.0
0.0
6.0
